In [1]:
import pandas as pd
from geopy.distance import great_circle

In [2]:
hospitals = pd.read_csv('df_geo.csv')

In [3]:
hospitals.head()

,Unnamed: 0,nazwa,adres,telefon,geo
0,0,WIELOSPECJALISTYCZNY SZPITAL MIEJSKI IM. JÓZEF...,"POZNAŃ 61-285, UL. SZWAJCARSKA 3",61 873 90 000,"('52.3895197', '16.995482')"
1,1,WOJEWÓDZKI SZPITAL ZESPOLONY IM. DR. ROMANA OS...,"KONIN 62-504, UL. SZPITALNA 45",63 240 43 000,"('52.1913128', '18.2331492')"
2,2,WOJEWÓDZKI SZPITAL ZESPOLONY IM.LUDWIKA PERZYN...,"KALISZ 62-800, UL. POZNAŃSKA 79",62 765 13 56,"('51.76522785', '18.0663836371474')"
3,3,SZPITAL POWIATOWY IM. ROMANA DREWSA W CHODZIEŻY,ŻEROMSKIEGO 29 64-800 CHODZIEŻ,67 282 92 21,"('52.9880523', '16.9157074')"
4,4,SAMODZIELNY PUBLICZNY ZAKŁAD OPIEKI ZDROWOTNEJ...,PODUCHOWNE 1 62-700 TUREK,63 280 55 08,"('52.01025685', '18.4908246978452')"


In [12]:
def to_tuple(a):
    if a == 'a':
        return ('a','a')
    return tuple([float(x.replace(',','').strip("'")) for x in a.replace('(','').replace(')','').split()])

In [7]:
hospitals = hospitals.drop(columns=['Unnamed: 0'])

In [8]:
hospitals.head(3)

,nazwa,adres,telefon,geo
0,WIELOSPECJALISTYCZNY SZPITAL MIEJSKI IM. JÓZEF...,"POZNAŃ 61-285, UL. SZWAJCARSKA 3",61 873 90 000,"('52.3895197', '16.995482')"
1,WOJEWÓDZKI SZPITAL ZESPOLONY IM. DR. ROMANA OS...,"KONIN 62-504, UL. SZPITALNA 45",63 240 43 000,"('52.1913128', '18.2331492')"
2,WOJEWÓDZKI SZPITAL ZESPOLONY IM.LUDWIKA PERZYN...,"KALISZ 62-800, UL. POZNAŃSKA 79",62 765 13 56,"('51.76522785', '18.0663836371474')"


In [9]:
hospitals['geo_lat'] = 0

In [10]:
hospitals['geo_lon'] = 0

In [13]:
hospitals.geo_lat = hospitals.geo.apply(lambda x: to_tuple(x)[0])

In [14]:
hospitals.geo_lon = hospitals.geo.apply(lambda x: to_tuple(x)[1])

In [15]:
hospitals.head(3)

,nazwa,adres,telefon,geo,geo_lat,geo_lon
0,WIELOSPECJALISTYCZNY SZPITAL MIEJSKI IM. JÓZEF...,"POZNAŃ 61-285, UL. SZWAJCARSKA 3",61 873 90 000,"('52.3895197', '16.995482')",52.3895,16.9955
1,WOJEWÓDZKI SZPITAL ZESPOLONY IM. DR. ROMANA OS...,"KONIN 62-504, UL. SZPITALNA 45",63 240 43 000,"('52.1913128', '18.2331492')",52.1913,18.2331
2,WOJEWÓDZKI SZPITAL ZESPOLONY IM.LUDWIKA PERZYN...,"KALISZ 62-800, UL. POZNAŃSKA 79",62 765 13 56,"('51.76522785', '18.0663836371474')",51.7652,18.0664


In [16]:
my_loc = (hospitals.geo_lat.loc[0],hospitals.geo_lon.loc[0])

In [18]:
hospitals_clear = hospitals.filter(hospitals.geo != 'a')

In [19]:
len(hospitals_clear)

162

In [20]:
len(hospitals)

162

In [26]:
hospitals.to_csv('df_coords.csv')

In [35]:
hospitals_clear = hospitals.filter(hospitals.loc[hospitals.geo == 'a'])

In [45]:
hospitals_clear = hospitals.drop(hospitals.loc[hospitals.geo == 'a'].index)

In [46]:
hospitals_clear.reset_index(drop=True,inplace=True)

In [48]:
hospitals_clear = hospitals_clear.drop(columns=['geo'])

In [49]:
hospitals_clear.head()

,nazwa,adres,telefon,geo_lat,geo_lon
0,WIELOSPECJALISTYCZNY SZPITAL MIEJSKI IM. JÓZEF...,"POZNAŃ 61-285, UL. SZWAJCARSKA 3",61 873 90 000,52.3895,16.9955
1,WOJEWÓDZKI SZPITAL ZESPOLONY IM. DR. ROMANA OS...,"KONIN 62-504, UL. SZPITALNA 45",63 240 43 000,52.1913,18.2331
2,WOJEWÓDZKI SZPITAL ZESPOLONY IM.LUDWIKA PERZYN...,"KALISZ 62-800, UL. POZNAŃSKA 79",62 765 13 56,51.7652,18.0664
3,SZPITAL POWIATOWY IM. ROMANA DREWSA W CHODZIEŻY,ŻEROMSKIEGO 29 64-800 CHODZIEŻ,67 282 92 21,52.9881,16.9157
4,SAMODZIELNY PUBLICZNY ZAKŁAD OPIEKI ZDROWOTNEJ...,PODUCHOWNE 1 62-700 TUREK,63 280 55 08,52.0103,18.4908


In [50]:
hospitals.to_csv('df_coords.csv')

In [51]:
my_loc = (52.34, 18.45)

In [52]:
def get_distance(from_coords, lat, lon):
    return great_circle(from_coords,(lat, lon)).kilometers
    

In [74]:
def get_closest(frame, position):
    distances = pd.DataFrame(frame.apply(lambda x: get_distance(position,x['geo_lat'],x['geo_lon']), axis=1))
    return frame.loc[distances.min().index.values]

In [76]:
get_closest(hospitals_clear,my_loc)

,nazwa,adres,telefon,geo_lat,geo_lon
0,WIELOSPECJALISTYCZNY SZPITAL MIEJSKI IM. JÓZEF...,"POZNAŃ 61-285, UL. SZWAJCARSKA 3",61 873 90 000,52.3895,16.9955


In [77]:
hospitals_clear.to_csv('df_coords_clear.csv')